In [20]:
import pandas as pd
import numpy as np
import re

In [21]:
df = pd.read_csv('immoscout24.csv', delimiter='|')

In [22]:
df.to_clipboard()

In [23]:
df.columns

Index(['Page', 'Type', 'Price', 'Address', 'Description', 'Weblink'], dtype='object')

In [24]:
df.to_clipboard()

In [25]:
df.iloc[4, 1]

'3 Zimmer,\xa081m², CHF 2’610.– '

In [26]:
# Function to extract values from 'Type' attribute and store in new columns
def extract_values(row):
    parts = re.split(',\s|,\xa0', row['Type'])  # Split by comma followed by space or non-breaking space
    print(parts)
    rooms = None
    size = None
    price2 = None
    if len(parts) == 1:
        price2 = parts[0]
    if len(parts) == 2:
        size = parts[0]
        price2 = parts[1]
    elif len(parts) == 3:
        rooms = parts[0]
        size = parts[1]
        price2 = parts[2]
    return pd.Series([rooms, size, price2], index=['rooms', 'size', 'price2'])

# Apply the function to each row of the DataFrame
df[['rooms', 'size', 'price2']] = df.apply(extract_values, axis=1)


# Function to move 'size' values containing 'Zimmer' to 'rooms' attribute
def move_to_rooms(row):
    if row['size'] is not None and 'zimmer' in row['size'].lower():
        row['rooms'], row['size'] = row['size'], None
    return row

# Apply the function to each row of the DataFrame
df = df.apply(move_to_rooms, axis=1)

print(df)

['4.5 Zimmer', '107m²', 'CHF 3’082.– ']
['3.5 Zimmer', '77m²', 'CHF 2’443.– ']
['4.5 Zimmer', '150m²', 'CHF 5’850.– ']
['5.5 Zimmer', 'CHF 5’500.– ']
['3 Zimmer', '81m²', 'CHF 2’610.– ']
['1.5 Zimmer', '25m²', 'CHF 1’710.– ']
['3.5 Zimmer', '111m²', 'CHF 3’839.– ']
['3.5 Zimmer', '65m²', 'CHF 3’190.– ']
['2.5 Zimmer', '109m²', 'CHF 4’280.– ']
['3.5 Zimmer', '75m²', 'CHF 3’300.– ']
['3.5 Zimmer', '71m²', 'CHF 2’590.– ']
['2.5 Zimmer', '56m²', 'CHF 4’095.– ']
['1.5 Zimmer', '56m²', 'CHF 3’020.– ']
['3.5 Zimmer', '140m²', 'CHF 6’790.– ']
['4.5 Zimmer', '109m²', 'CHF 5’990.– ']
['3.5 Zimmer', '88m²', 'CHF 4’990.– ']
['3.5 Zimmer', '81m²', 'CHF 5’983.– ']
['4.5 Zimmer', '121m²', 'CHF 5’125.– ']
['2.5 Zimmer', '95m²', 'CHF 3’000.– ']
['1 Zimmer', '37m²', 'CHF 1’630.– ']
['4.5 Zimmer', '108m²', 'CHF 4’630.– ']
['5.5 Zimmer', '133m²', 'CHF 5’690.– ']
['3.5 Zimmer', '99m²', 'CHF 3’590.– ']
['2.5 Zimmer', '87m²', 'CHF 2’595.– ']
['4.5 Zimmer', '122m²', 'CHF 4’190.– ']
['5 Zimmer', '156m²', 'CHF 

In [27]:
# One hot encoding for rooms

# Function to check and update 'rooms' attribute based on 'Description'
def update_rooms(row):
    if pd.isna(row['rooms']):
        description = row['Description'].lower()
        if 'studio' in description:
            row['rooms'] = '1 Zimmer'
        elif '1 bedroom' in description:
            row['rooms'] = '1 Zimmer'
        elif '2 bedroom' in description:
            row['rooms'] = '2 Zimmer'
    return row

# Apply the function to each row of the DataFrame
df = df.apply(update_rooms, axis=1)

# Filter out None values from 'rooms' attribute
filtered_rooms = [room for room in df['rooms'] if room is not None]

# Extract unique values
unique_rooms = list(set(filtered_rooms))

# Sort unique values
unique_rooms.sort()

# Create new attributes for each unique value
for room_type in unique_rooms:
    df[room_type] = df['rooms'].apply(lambda x: 1 if x == room_type else 0)

print(df)

     Page                             Type          Price  \
0       2  4.5 Zimmer, 107m², CHF 3’082.–    CHF 3’082.–    
1       2   3.5 Zimmer, 77m², CHF 2’443.–    CHF 2’443.–    
2       2  4.5 Zimmer, 150m², CHF 5’850.–    CHF 5’850.–    
3       2         5.5 Zimmer, CHF 5’500.–    CHF 5’500.–    
4       2     3 Zimmer, 81m², CHF 2’610.–    CHF 2’610.–    
..    ...                              ...            ...   
640    43   1.5 Zimmer, 34m², CHF 2’120.–    CHF 2’120.–    
641    43     2 Zimmer, 50m², CHF 2’430.–    CHF 2’430.–    
642    43     2 Zimmer, 49m², CHF 2’320.–    CHF 2’320.–    
643    43         3.5 Zimmer, CHF 2’570.–    CHF 2’570.–    
644    43                       CHF 180.–      CHF 180.–    

                                         Address  \
0                  In der Looren 56, 8053 Zürich   
1    Kügeliloostrasse 99, Gebäude F, 8046 Zürich   
2                     Wirzenweid 27, 8053 Zürich   
3                          Lyrenweg, 8047 Zürich   
4      

In [28]:
df.to_clipboard()

In [29]:
# Column: 'Price'
for index, row in df.iterrows():
    price_str = df.at[index, 'Price'].strip()
    # Remove "CHF" and any non-numeric characters, replace thousands separator

    cleaned_str = re.sub(r'[^\d]', '', price_str.replace("’", ''))

    # Convert to integer
    try:
        df.at[index, 'Price'] = int(cleaned_str)
    except:
        df.at[index, 'Price'] = np.NAN

nan_count = df['Price'].isna().sum()
print(nan_count)
df.dropna(subset=['Price'], inplace=True)

# To verify if all have been removed
df[df['Price'].isna()]

4


,Page,Type,Price,Address,Description,Weblink,rooms,size,price2,1 Zimmer,...,20 Zimmer,3 Zimmer,3.5 Zimmer,4 Zimmer,4.5 Zimmer,5 Zimmer,5.5 Zimmer,6 Zimmer,6.5 Zimmer,7 Zimmer


In [30]:
# Custom function to edit addresses: remove values with additional comma
def edit_address(Address):
    # Split the address by commas
    parts = Address.split(', ')
    
    # Check if there are two commas
    if len(parts) == 3:
        # Join the parts at positions 0 and 2 to remove the part at position 1
        edited_address = ', '.join([parts[0], parts[2]])
        return edited_address
    
    # Return the original address if there are not two commas
    return Address

# Apply the function to edit addresses in the 'address' column
df['Address'] = df['Address'].apply(edit_address)

# 'Address' into 'address_short_1' and 'address_short_2' with conditions
def split_address(row):
    parts = row['Address'].split(', ')
    if len(parts) == 2:
        row['address_short_1'] = parts[0]
        row['address_short_2'] = parts[1]
    elif len(parts) == 1:
        row['address_short_1'] = None
        row['address_short_2'] = parts[0]
    return row

# Apply the function to each row of the DataFrame
df = df.apply(split_address, axis=1)


In [31]:
df.to_clipboard()

In [32]:
# Store size as float
df['size_num'] = df['size'].str.extract(r'(\d+)').astype(float)


In [33]:
df['size_num']

0      107.0
1       77.0
2      150.0
3        NaN
4       81.0
       ...  
640     34.0
641     50.0
642     49.0
643      NaN
644      NaN
Name: size_num, Length: 641, dtype: float64

In [34]:
# Rename columns to lowercase
df.rename(columns=lambda x: x.lower(), inplace=True)

# Remove the word 'Zimmer' from the 'rooms' column and convert to numeric
df['rooms'] = df['rooms'].str.replace(' Zimmer', '').astype(float)

In [35]:
# Save to file
df.to_csv('immoscout24_cleansed_3.csv', sep="|", index=False)